# Use Cohere Command R or Cohere Command R+ with Azure AI Search to Answer Questions Using Your Data

You can use Cohere `Command R` or Cohere `Command R+` models deployed in `Azure AI`and Azure ML to create advanced retrieval augmented generation (RAG) pipelines.

> Review the [documentation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-cohere-command) for the Cohere family of models at for AI Studio and for ML Studio for details on how to provision inference endpoints, regional availability, pricing and inference schema reference.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home)
* Create a project and resource group
* Select `Cohere Command R` or `Cohere Command R+`.

    > Notice that some models may not be available in all the regions in Azure AI and Azure Machine Learning. On those cases, you can create a workspace or project in the region where the models are available and then consume it with a connection from a different one. To learn more about using connections see [Consume models with connections](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deployments-connections)

* Deploy with "Pay-as-you-go"
* Follow the same steps for `Cohere-embed-v3-english`

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation [here](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-cohere-command) for model deployment and inference.

### Services

You will need to ensure the following services have been created in your Azure environment:
* Ensure you have created a search service. This can be done in the `Azure Portal` and more instructions can be found here: https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search **NOTE: You do not need to create the index, this will be done below**

* Create a Cohere `Embed` and `Command` endpoint in the `Azure AI Studio`. Instructions can be found here: https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-cohere-command 

## Example

The following example demonstrate how to create a RAG workflow that uses a `Cohere Command R` or `Cohere Command R+` model deployed in Azure AI and Azure ML. We will also leverage Azure AI Search to store our documents along with LangChain to orchestrate the process

### Install Dependencies

In [ ]:
! pip install --quiet cohere azure-search-documents azure-identity

In [91]:
import cohere
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchIndex,
)


## Constants
We will set the values for the keys and models that we will use in our RAG pipeline

Let's create instances of our Chat and Embed models deployed in Azure AI or Azure ML. Use the `langchain_cohere` package and configure it as follows:

- `embed_endpoint` and `command_endpoint`: Use the endpoint URL from your deployment. Include `/v1` at the end of the endpoint URL.

In [ ]:
# set constants
search_service_endpoint = "https://<resource>.search.windows.net"
key_credential = AzureKeyCredential("AZURE_SEARCH_ADMIN_KEY")
index_name = "AZURE_SEARCH_INDEX"  # this index does not have to be created yet

# Embed
azure_cohere_embed_endpoint = "https://<endpoint>.<region>.inference.ai.azure.com/v1"
azure_cohere_embed_key = "<key>"

# Command
azure_cohere_command_endpoint = "https://<endpoint>.<region>.inference.ai.azure.com/v1"
azure_cohere_command_key = "<key>"

## AI Search Index

We need to set up the index in our `Azure AI Search`

We will use the `Cohere` package as well and set the URL to be our Azure AI endpoint, which will serve as the embedding model for our index

In [ ]:
# Set up the embedding model to be used in the vector index
co_embed = cohere.ClientV2(
    base_url=azure_cohere_embed_endpoint, api_key=azure_cohere_embed_key
)

Lets create some sample data to add to our index

In [ ]:
docs_to_index = [
    {
        "hotelId": "1",
        "hotelName": "Fancy Stay",
        "description": "Best hotel in town if you like luxury hotels.",
        "category": "Luxury",
    },
    {
        "hotelId": "2",
        "hotelName": "Roach Motel",
        "description": "Cheapest hotel in town. Infact, a motel.",
        "category": "Budget",
    },
    {
        "hotelId": "3",
        "hotelName": "EconoStay",
        "description": "Very popular hotel in town.",
        "category": "Budget",
    },
    {
        "hotelId": "4",
        "hotelName": "Modern Stay",
        "description": "Modern architecture, very polite staff and very clean. Also very affordable.",
        "category": "Luxury",
    },
    {
        "hotelId": "5",
        "hotelName": "Secret Point",
        "description": "One of the best hotel in town. The hotel is ideally located on the main commercial artery of the city in the heart of New York.",
        "category": "Boutique",
    },
]

## Create the embeddings
We will create a vector field for each of the hotel descriptions

In [ ]:
# embed each of the descriptions
# you will notice that Cohere has a field called "input_type" which can be set to "search_document", "search_query", "classification", or "clustering" depedning on the text you are embedding
for doc in docs_to_index:
    doc["descriptionVector"] = co_embed.embed(
        model="azureai",
        texts=[doc["description"]],
        input_type="search_document",  # the type of content being embedded. Can be one of "search_document, "search_query", "classification", "clustering", or "image"
        embedding_types=[
            "float"
        ],  # the format of the embeddings. Can be one or more of "float", "int8", "uint8", "binary",
    ).embeddings["float"][0]

## Create the index
Next we will create the index using the Azure SDK

In [ ]:
# create the search index in azure
index_client = SearchIndexClient(
    endpoint=search_service_endpoint, credential=key_credential
)
fields = [
    SimpleField(
        name="hotelId",
        type=SearchFieldDataType.String,
        key=True,
        sortable=True,
        filterable=True,
        facetable=True,
    ),
    SearchableField(name="hotelName", type=SearchFieldDataType.String),
    SearchableField(name="description", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String, filterable=True),
    SearchField(
        name="descriptionVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1024,
        vector_search_profile_name="myHnswProfile",
    ),
]

# Configure the vector search configuration
vector_search = VectorSearch(
    algorithms=[HnswAlgorithmConfiguration(name="myHnsw")],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        )
    ],
)

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)
result = index_client.create_or_update_index(index)
print(f" {result.name} created")

## Insert the embeddings
Finally, we will add the data into our vector index

In [ ]:
search_client = SearchClient(
    endpoint=search_service_endpoint, index_name=index_name, credential=key_credential
)

In [ ]:
search_client.upload_documents(documents=docs_to_index)

## Vector Search
We will create a small wrapper function to embed our query and search using Azure AI similarity search

In [ ]:
# Search function to encode the query and search the index
def search(query):
    """
    Searches for documents based on the given query.

    Args:
        query (str): The search query.

    Returns:
        list: A list of search results.
    """
    query_embedding = co_embed.embed(
        model="azureai",
        texts=[query],
        input_type="search_query",  # the type of content being embedded. Can be one of "search_document, "search_query", "classification", "clustering", or "image"
        embedding_types=[
            "float"
        ],  # the format of the embeddings. Can be one or more of "float", "int8", "uint8", "binary",
    ).embeddings["float"][0]

    # Azure AI search requires a vector query
    vector_query = VectorizedQuery(
        vector=query_embedding, k_nearest_neighbors=3, fields="descriptionVector"
    )

    search_results = search_client.search(
        search_text=None,
        vector_queries=[vector_query],
        select=["hotelName", "description", "category"],
    )

    return search_results

In [ ]:
# Test our search function
for r in search("luxury hotel"):
    print(r)

### Create our Chat function
Next, we will create a basic chat function using the `Chat` Cohere endpoint 

Cohere's `chat endpoint` can accept documents directly and will return a grounded answer that includes citations against those documents.

No prompt is needed since the `Cohere` model will automatically use a RAG prompt when documents are passed in.

In [ ]:
co_chat = cohere.ClientV2(
    base_url=azure_cohere_command_endpoint, api_key=azure_cohere_command_key
)

In [ ]:
def ask(question):
    """
    Ask a question to the chatbot, expecting a dictionary with 'question' and 'documents'.

    Args:
        inputs (dict): A dictionary containing 'question' and 'documents'.

    Returns:
        str: The response from the chatbot.
    """
    search_results = search(question)

    # select category, description, and hotelName from the search results
    documents = [
        {
            "id": f"{index}",
            "data": {
                "category": result["category"],
                "description": result["description"],
                "hotelName": result["hotelName"],
            },
        }
        for index, result in enumerate(search_results)
    ]

    response = co_chat.chat(
        model="azureai",
        messages=[{"role": "user", "content": question}],
        documents=documents,
    )

    return response

In [ ]:
res = ask("Whats a good hotel close to city center?")
print(res)

## Clean the results
We can retrieve the answer and the citations from the response

In [ ]:
def pretty_text(text, citations):

    # Process each citation in reverse order to prevent index shifting
    text_with_citations = ""
    text_start_index = 0
    for citation in citations:
        doc_ids_str = ", ".join([source.id for source in citation.sources])
        citated_text = text[citation.start : citation.end]
        # Bold the citation text and add document ids as superscript
        cited_text_with_ids = f"**{citated_text}**^({doc_ids_str})"
        text_with_citations = (
            text[text_start_index : citation.start] + cited_text_with_ids
        )
        text_start_index = citation.end

    text_with_citations += text[text_start_index:]

    return text_with_citations

In [ ]:
pretty_text_output = pretty_text(res.message.content[0].text, res.message.citations)

In [ ]:
print(pretty_text_output)

## Aditional resources

Here are some additional reference:

* [Plan and manage costs](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)

* [Learn more about the Cohere SDK](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-cohere-command)